# Freeman decomposition evaluation

In this notebook we compare results of the new implementation of the
Freeman decomposition with the C legacy version of PolSARpro.  
It assumes the user has a working installation of PolSARpro and has followed the 
instructions in the README.md file.

## Scope of the work
- Adaptation of the Freeman 3 component decomposition code to a permissive open-source
license.
- Optimization of the code for pythonic capabilities (e.g., vectorization, parallelization, full
use of NumPy).
- Removal of any unused code lines to streamline and enhance maintainability

In [1]:
# run only once to enable old lib
import sys
sys.path.remove("/polsarpro-dev")
sys.path.append("../../old_files/") 

In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
from pathlib import Path
from polsarpro.io import read_PSP_bin, read_T3
from polsarpro.decompositions import freeman, freeman_dask
from pathlib import Path
import os
import matplotlib.pyplot as plt
import numpy as np
from spectral.io.envi import save_image

# change to your local C-PolSARpro install dir
c_psp_dir = "/home/c_psp/Soft/bin/"
os.environ["PATH"]+=os.pathsep+f"{c_psp_dir}/data_process_sngl/"
os.environ["PATH"]+=os.pathsep+f"{c_psp_dir}/data_convert/"

# change to your data paths
input_test_dir = Path("/data/psp/input/freeman_decomposition/")
output_test_dir = Path("/data/res/freeman_cpsp")
# output_test_dir = Path("/data/psp/output/freeman_decomposition/out")

## Run the C-version on some test data

In [ ]:
from polsarpro.devtools import parse_psp_parameter_string
import os

input_str = f"""id: {input_test_dir}
od: {output_test_dir}
iodf: T3
nwr: 7
nwc: 7
ofr: 0
ofc: 0
fnr: 18432
fnc: 1248
errf: /tmp"""

result = parse_psp_parameter_string(input_str)
os.system(f"freeman_decomposition.exe {result}")

## Load T3 coherency matrices and C-PSP Freeman outputs

In [ ]:
T3 = read_T3(input_test_dir)

file_odd = output_test_dir / "Freeman_Odd.bin"
ps_c = read_PSP_bin(file_odd)
file_dbl = output_test_dir / "Freeman_Dbl.bin"
pd_c = read_PSP_bin(file_dbl)
file_vol = output_test_dir / "Freeman_Vol.bin"
pv_c = read_PSP_bin(file_vol)

## Apply the dask implementation (multi-thread)

In [ ]:
ps_da, pd_da, pv_da = freeman_dask(T3, "T3", boxcar_size=[7, 7])

# write outputs
save_image(hdr_file=output_test_dir / "Freeman_Odd_py.hdr", image=ps_da, force=True)
save_image(hdr_file=output_test_dir / "Freeman_Dbl_py.hdr", image=pd_da, force=True)
save_image(hdr_file=output_test_dir / "Freeman_Vol_py.hdr", image=pv_da, force=True)

## Apply the numpy implementation (single-thread, only for reference)

In [ ]:
ps, pd, pv = freeman(T3, "T3", boxcar_size=[7, 7])

## Numeric and visual evaluation

In [ ]:
print(f"difference between dask and normal implementation for odd {np.abs(ps - ps_da).sum()}, double {np.abs(pd - pd_da).sum()}, and volume {np.abs(pv - pv_da).sum()}")

In [ ]:
var_name = "P_v"
var_p = pv_da
var_c = pv_c

plt.figure(figsize=(10, 10))
plt.imshow(var_p[::8][1000:1500], interpolation="none", vmax=1)
plt.colorbar(fraction=0.046, pad=0.04)
plt.title(f"${var_name}$ python")

plt.figure(figsize=(10, 10))
plt.imshow(var_c[::8][1000:1500], interpolation="none", vmax=1)
plt.colorbar(fraction=0.046, pad=0.04)
plt.title(f"${var_name}$ C")

np.nanmean(((np.abs(var_p - var_c) / np.abs(var_c))))

In [ ]:
var_name = "P_s"
var_p = ps_da
var_c = ps_c

plt.figure(figsize=(10, 10))
plt.imshow(var_p[::8][1000:1500], interpolation="none", vmax=1)
plt.colorbar(fraction=0.046, pad=0.04)
plt.title(f"${var_name}$ python")

plt.figure(figsize=(10, 10))
plt.imshow(var_c[::8][1000:1500], interpolation="none", vmax=1)
plt.colorbar(fraction=0.046, pad=0.04)
plt.title(f"${var_name}$ C")

np.nanmean(((np.abs(var_p - var_c) / np.abs(var_c))))

In [ ]:
var_name = "P_d"
var_p = pd_da
var_c = pd_c

plt.figure(figsize=(10, 10))
plt.imshow(var_p[::8][1000:1500], interpolation="none", vmin=0, vmax=1)
plt.colorbar(fraction=0.046, pad=0.04)
plt.title(f"${var_name}$ python")

plt.figure(figsize=(10, 10))
plt.imshow(var_c[::8][1000:1500], interpolation="none", vmax=1)
plt.colorbar(fraction=0.046, pad=0.04)
plt.title(f"${var_name}$ C")

np.nanmean(((np.abs(var_p - var_c) / np.abs(var_c))))